In [1]:
import os 
import json
import psycopg2

In [2]:
#establishing connection to db
conn = psycopg2.connect("host=localhost port=5432 dbname=pid-portal user=postgres password=passwd") 
cur = conn.cursor()

In [3]:
cur.execute('SELECT * FROM "User"') #just simple select to check if connection is working
cur.fetchone()

# Vehicle positions


In [ ]:
def digest(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #df = pd.read_json(file, lines=True)
    return data


In [ ]:
entries = [digest(entry) for entry in os.scandir('../../data/') if entry.is_file()]

In [ ]:
entries[0][2]['features'][4]['properties']


In [ ]:
i = 0
for line in entries[0]:
    for data in line['features']:
        cur.execute("""
                INSERT INTO public."Vehicle_position"(
                route_id, "timestamp", bearing, calculated_delay, speed, longitude, latitude, vehicle_id, trip_id, is_canceled)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """,
               (data['properties']['trip']['gtfs_route_id'],data['properties']['trip']['start_timestamp'], 
                data['properties']['last_position']['bearing'], data['properties']['last_position']['delay'],
                data['properties']['last_position']['speed'], data['properties']['last_position']['lng'], data['properties']['last_position']['lat'],
                data['properties']['trip']['cis_vehicle_registration_number'], data['properties']['trip']['gtfs_trip_id'], data['properties']['last_position']['is_canceled']
               ))
        #filling the vehicle table from the position data
        cur.execute("""
        INSERT INTO public."Vehicle"(
	vehicle_id, vehicle_type, agency)
	VALUES (%s, %s, %s)
    ON CONFLICT (vehicle_id) DO UPDATE 
  SET vehicle_type = excluded.vehicle_type, 
  agency = excluded.agency
    ;
        """,
            (data['properties']['trip']['cis_vehicle_registration_number'], data['properties']['trip']['vehicle_type'], data['properties']['trip']['cis_agency_name'])
                   )
        i += 1
print('Executed {} entries for table Vehicle_position'.format(i))

In [ ]:
conn.commit()

# Stops

In [ ]:
def get_bool_from_int_for_wheelchair(bool_val):
    if bool_val == 1:
        return "1"
    else:
        return "0"

In [ ]:
def digest_stops(file):
    f = open(file, 'r')
    #data = [json.loads(line) for line in open(file, 'r')]
    data = json.load(f)
    return data

In [ ]:
entries = [digest_stops(entry) for entry in os.scandir('/mnt/d/data_diplomka/stops') if entry.is_file()]

In [ ]:
entries[0]['features'][5]['properties']

In [ ]:
i = 0
for a in entries[0]['features']:
    
    cur.execute("""
    INSERT INTO public."Stop"(
	stop_id, name, latitude, longitude, zone_id, wheelchair_acc)
	VALUES (%s, %s, %s, %s, %s, %s);
    """,(a['properties']['stop_id'], a['properties']['stop_name'], a['properties']['stop_lat'], a['properties']['stop_lon'], a['properties']['zone_id'], 
        get_bool_from_int_for_wheelchair(a['properties']['wheelchair_boarding'])
        ))
    i += 1
print('Executed {} entries for table Stop'.format(i))

In [ ]:
conn.commit()

# Routes

In [ ]:
def digest_routes(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [ ]:
entries = [digest_routes(entry) for entry in os.scandir('/mnt/d/data_diplomka/routes_test') if entry.is_file()]

In [ ]:
entries[1][0]['features'][4]['properties']

In [ ]:
i = 0
for route in entries:
    for point in route[0]['features']:
        cur.execute("""
        INSERT INTO public."Route"(
        route_id, "order", distance, latitude, longitude, stop_id)
        VALUES (%s, %s, %s, %s, %s, %s);
        """, 
                    (point['properties']['shape_id'], point['properties']['shape_pt_sequence'], point['properties']['shape_dist_traveled'], point['properties']['shape_pt_lat'],
                    point['properties']['shape_pt_lon'], '')
                   )
        i += 1
        #this will update the route with stops by finding route points that are within 10cm from stop location
cur.execute("""
        UPDATE "Route"
        SET stop_id=subquery.stop_id
            FROM (
            select A.stop_id, B.route_id, B.order from "Stop" as A, "Route" as B where calculate_distance(A.latitude, A.longitude, B.latitude,B.longitude, 'K') < 0.01
            ) AS subquery
        WHERE "Route".route_id=subquery.route_id and "Route".order=subquery.order;
        """)
        
print('Executed {} entries for stops'.format(i))

In [ ]:
conn.commit()

# Trip

In [ ]:
def digest_trips(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [ ]:
entries = [digest_trips(entry) for entry in os.scandir('/mnt/d/data_diplomka/trips') if entry.is_file()]

In [ ]:
entries[0][1]

In [ ]:
for file in entries:
    for entry in file:
        cur.execute("""
        INSERT INTO public."Trip"(
	trip_id, route_id, shape_id, direction, bikes_allowed, headsign)
	VALUES (%s, %s, %s, %s, %s, %s);""", 
                    (entry['trip_id'], entry['route_id'], entry['shape_id'], get_bool_from_int_for_wheelchair(entry['direction_id']), get_bool_from_int_for_wheelchair(entry['bikes_allowed']),
                    entry['trip_headsign'])
                    )

In [ ]:
conn.commit()

# Stop Times

In [4]:
def digest_st_times(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [5]:
entries = [digest_st_times(entry) for entry in os.scandir('/mnt/c/data_diplomka/stop_times') if entry.is_file()]

In [6]:
entries[1][0]

{'arrival_time': '10:00:05',
 'arrival_time_seconds': None,
 'departure_time': '10:00:05',
 'departure_time_seconds': None,
 'drop_off_type': '0',
 'pickup_type': '0',
 'shape_dist_traveled': 0,
 'stop_headsign': '',
 'stop_id': 'U1000Z102P',
 'stop_sequence': 1,
 'timepoint': None,
 'trip_id': '993_960_190707'}

In [7]:
for file in entries:
    for entry in file:
        cur.execute("""
        INSERT INTO public.stop_time(
	trip_id, stop_id, sequence, arival_time, departure_time)
	VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (trip_id, stop_id, sequence) DO UPDATE 
  SET trip_id = excluded.trip_id, 
  stop_id = excluded.stop_id,
  sequence = excluded.sequence,
  arival_time = excluded.arival_time,
  departure_time = excluded.departure_time
  ;""", 
                    (entry['trip_id'], entry['stop_id'], entry['stop_sequence'], entry['arrival_time'],
                    entry['departure_time'])
                    )

In [8]:
conn.commit() 

In [9]:
entries = []